In [1]:
"""
EDD Report Summary Generator - ULTRA FAST VERSION
Target: 16 min → 1-2 min
Uses xlsxwriter for speed, outputs separate summary file

✅ UPDATE: Added section-wise timing (NO logic/process changed)
✅ UPDATE 2: Faster Excel read via Calamine (fallback to openpyxl) (NO logic/process changed)
✅ UPDATE 3: WH Destination + Blank NDR_Remark (Zone-wise + Week-wise) in Summary (NO existing logic changed)
✅ UPDATE 4: Show WH Destination TOTAL count (Zone-wise + Week-wise) under WHD block (NO existing logic changed)
"""

import pandas as pd
import numpy as np
from datetime import time
import xlsxwriter
import os
import time as _time  # ✅ for timing


# ===================== CONFIG =====================
INPUT_FILE = "EDD Report.xlsx"
OUTPUT_FILE = "EDD_Summary_Report.xlsx"
SOURCE_SHEET = "Query result"

NEEDED_COLS = [
    "EDD_Date", "PICKUP_CHLN_DATE", "Reached At Destination", "DLY_Date",
    "TAT_DAYS", "BKG_Zone", "TPTR_Mode", "CN_Current_Status", "BUSINESS_TYPE",
    "NDR_Remark"  # ✅ NEW (required for WH Destination blank NDR logic)
]

print("⏳ Starting optimized processing...")

# ===================== TIMING HELPERS (NEW) =====================
_T0 = _time.perf_counter()
_last = _T0

def _tick(label: str):
    """Print time taken since last tick + total elapsed"""
    global _last
    now = _time.perf_counter()
    step = now - _last
    total = now - _T0
    print(f"⏱️  {label:<35} | step: {step:7.2f}s | total: {total:7.2f}s")
    _last = now


# ===================== SINGLE FAST READ (UPDATED ENGINE) =====================
print("📖 Reading Excel file...")

def _read_excel_fast():
    # Keep EXACT same behavior as before; just engine selection changes.
    # Try calamine first (fast), fallback to openpyxl.
    read_kwargs = dict(
        io=INPUT_FILE,
        sheet_name=SOURCE_SHEET,
        usecols=lambda x: str(x).strip() in NEEDED_COLS,  # Filter during read (unchanged logic)
        dtype={
            'BKG_Zone': 'category',
            'TPTR_Mode': 'category',
            'CN_Current_Status': 'category',
            'BUSINESS_TYPE': 'category'
        }
    )

    # 1) Try Calamine (fast)
    try:
        print("   ⚡ Trying engine='calamine' ...")
        df_local = pd.read_excel(engine="calamine", **read_kwargs)
        return df_local, "calamine"
    except Exception as e:
        print(f"   ⚠️ Calamine not available/failed, fallback to openpyxl. Reason: {e}")

    # 2) Fallback Openpyxl (default)
    df_local = pd.read_excel(engine="openpyxl", **read_kwargs)
    return df_local, "openpyxl"


df, _engine_used = _read_excel_fast()
df.columns = df.columns.astype(str).str.strip()

# ✅ Safety: if column missing in some file, create it to avoid crash (no impact if it exists)
if "NDR_Remark" not in df.columns:
    df["NDR_Remark"] = np.nan

print(f"   ✓ Loaded {len(df):,} rows (engine={_engine_used})")
_tick("Excel read")


# ===================== FAST DATE CONVERSION =====================
print("📅 Processing dates...")
date_cols = ["EDD_Date", "PICKUP_CHLN_DATE", "Reached At Destination", "DLY_Date"]
for col in date_cols:
    if col in df.columns:
        df[col] = pd.to_datetime(df[col], errors="coerce")

df = df.dropna(subset=["EDD_Date"])
_tick("Date conversion + dropna(EDD_Date)")


# ===================== VECTORIZED CALCULATIONS =====================
print("🔢 Calculating metrics...")

# NEW EDD DATE
df["NEW_EDD_DATE"] = df["PICKUP_CHLN_DATE"] + pd.to_timedelta(df["TAT_DAYS"] - 1, unit="D")

# ON TIME ARRIVAL - Vectorized
valid_arrival = (
    df["Reached At Destination"].notna() &
    df["NEW_EDD_DATE"].notna()
)
cond1 = df["Reached At Destination"] <= df["NEW_EDD_DATE"]
cond2 = (
    (df["Reached At Destination"].dt.normalize() == df["EDD_Date"].dt.normalize()) &
    (df["Reached At Destination"].dt.hour < 12)
)
df["ON_TIME_ARRIVAL"] = np.where(valid_arrival & (cond1 | cond2), "Yes", "No")

# ON TIME DELIVERY - Vectorized
df["ON_TIME_DELIVERY"] = np.where(
    df["DLY_Date"].notna() & (df["DLY_Date"] <= df["EDD_Date"]),
    "Yes", "No"
)

# WEEK LABEL
df["Week_Label"] = "W-" + df["EDD_Date"].dt.isocalendar().week.astype(str)

_tick("Vectorized metrics (EDD/OTA/OTD/Week_Label)")


# ===================== PRE-AGGREGATIONS (Single Pass) =====================
print("📊 Aggregating data...")

weekly_total = df.groupby("Week_Label", observed=True).size().sort_index()
weeks = weekly_total.index.tolist()

zone_total = df.groupby(["BKG_Zone", "Week_Label"], observed=True).size()
bt_counts = df.groupby(["BKG_Zone", "BUSINESS_TYPE", "Week_Label"], observed=True).size()
mode_counts = df.groupby(["BKG_Zone", "TPTR_Mode", "Week_Label"], observed=True).size()

ota_counts = df[df["ON_TIME_ARRIVAL"] == "Yes"].groupby(
    ["BKG_Zone", "TPTR_Mode", "Week_Label"], observed=True
).size()

otd_counts = df[df["ON_TIME_DELIVERY"] == "Yes"].groupby(
    ["BKG_Zone", "TPTR_Mode", "Week_Label"], observed=True
).size()

status_counts = df.groupby(["BKG_Zone", "CN_Current_Status", "Week_Label"], observed=True).size()

# ✅ WH Destination denominator & blank NDR numerator (Zone-wise + Week-wise)
_status_str = df["CN_Current_Status"].astype("string")
whd_df = df[_status_str.str.strip().eq("Ware house Destination")]

whd_total = whd_df.groupby(["BKG_Zone", "Week_Label"], observed=True).size()

ndr_str = whd_df["NDR_Remark"].astype("string")
whd_blank_mask = whd_df["NDR_Remark"].isna() | (ndr_str.str.strip().eq(""))
whd_blank = whd_df[whd_blank_mask].groupby(["BKG_Zone", "Week_Label"], observed=True).size()

zones = sorted(df["BKG_Zone"].dropna().unique().tolist())
all_modes = sorted(df["TPTR_Mode"].dropna().unique().tolist())
all_statuses = sorted(df["CN_Current_Status"].dropna().unique().tolist())
all_business_types = sorted(df["BUSINESS_TYPE"].dropna().unique().tolist())

_tick("Groupbys + unique lists (+ WHD NDR blank)")


# ===================== HELPER FUNCTIONS =====================
def get_series(key, series_data):
    """Fast series extraction with reindex"""
    try:
        if isinstance(key, tuple):
            return series_data.xs(key).reindex(weeks, fill_value=0)
        return series_data.reindex(weeks, fill_value=0)
    except KeyError:
        return pd.Series(0, index=weeks)

def format_pct(counts, totals):
    """Vectorized percentage formatting"""
    counts = counts.reindex(weeks, fill_value=0).values
    totals = totals.reindex(weeks, fill_value=0).values

    result = []
    for c, t in zip(counts, totals):
        if t > 0:
            result.append(f"{round((c/t)*100, 2)}% ({int(c)})")
        else:
            result.append("0% (0)")
    return result

def format_count(series_like):
    """Count formatting (Zone+Week) -> int list"""
    return series_like.reindex(weeks, fill_value=0).astype(int).tolist()

_tick("Helper functions defined")


# ===================== BUILD SUMMARY DATA =====================
print("📝 Building summary...")

rows_data = []  # List of (row_name, values_list, level)

# Picked Volume (Level 0 - always visible)
rows_data.append(("Picked Volume", weekly_total.reindex(weeks).tolist(), 0))

# ✅ optional: track zone loop time too (still no logic changes)
_zone_t0 = _time.perf_counter()

for zone in zones:
    z_tot = get_series((zone,), zone_total)

    # Zone header (Level 1 - collapsible)
    rows_data.append((f"Picked Vol. Zone {zone} %", format_pct(z_tot, weekly_total), 1))

    # ✅ WH Destination + Blank NDR_Remark (zone+week)
    whd_z_tot = get_series((zone,), whd_total)       # denominator (WHD total for that zone-week)
    whd_z_blank = get_series((zone,), whd_blank)     # numerator (WHD with blank NDR for that zone-week)

    rows_data.append(("WHD NDR Remark Blank", [""] * len(weeks), 2))

    # ✅ NEW: show total WH Destination count (Zone+Week)
    rows_data.append(("   WH Destination Total (count)", format_count(whd_z_tot), 3))

    # Existing: blank % with numerator count in brackets
    rows_data.append(("   Blank NDR % (count)", format_pct(whd_z_blank, whd_z_tot), 3))

    # BUSINESS TYPE BLOCK
    rows_data.append(("BUSINESS TYPE BREAKDOWN", [""] * len(weeks), 2))
    for bt in all_business_types:
        bt_ser = get_series((zone, bt), bt_counts)
        rows_data.append((f"   {bt}", format_pct(bt_ser, z_tot), 3))

    # TPTR MODE BLOCKS
    for mode in all_modes:
        m_ser = get_series((zone, mode), mode_counts)
        rows_data.append((f"TPTR Mode {mode}", format_pct(m_ser, z_tot), 2))

        ota_ser = get_series((zone, mode), ota_counts)
        rows_data.append((f"{mode} On Time Arrival", format_pct(ota_ser, m_ser), 3))

        otd_ser = get_series((zone, mode), otd_counts)
        rows_data.append((f"{mode} On Time Delivery", format_pct(otd_ser, m_ser), 3))

    # CN STATUS BLOCK
    rows_data.append(("CN Status Breakdown", [""] * len(weeks), 2))
    for status in all_statuses:
        s_ser = get_series((zone, status), status_counts)
        rows_data.append((f"   {status}", format_pct(s_ser, z_tot), 3))

_zone_t1 = _time.perf_counter()
_tick(f"Build rows_data (zones loop: {(_zone_t1 - _zone_t0):.2f}s)")


# ===================== WRITE WITH XLSXWRITER (FAST!) =====================
print("💾 Writing Excel file...")

workbook = xlsxwriter.Workbook(OUTPUT_FILE, {'constant_memory': True})
worksheet = workbook.add_worksheet('summary')

# --- Define Formats (once) ---
header_format = workbook.add_format({
    'bold': True,
    'font_color': 'white',
    'bg_color': '#1F4E78',
    'align': 'center',
    'valign': 'vcenter',
    'border': 1,
    'border_color': '#D9D9D9'
})

cell_format = workbook.add_format({
    'align': 'center',
    'valign': 'vcenter',
    'border': 1,
    'border_color': '#D9D9D9'
})

left_format = workbook.add_format({
    'align': 'left',
    'valign': 'vcenter',
    'border': 1,
    'border_color': '#D9D9D9'
})

# Level-based formats for visual hierarchy
level_formats = {
    0: workbook.add_format({'bold': True, 'bg_color': '#E2EFDA', 'align': 'left', 'border': 1}),
    1: workbook.add_format({'bold': True, 'bg_color': '#DDEBF7', 'align': 'left', 'border': 1}),
    2: workbook.add_format({'bold': True, 'bg_color': '#FCE4D6', 'align': 'left', 'border': 1}),
    3: workbook.add_format({'align': 'left', 'border': 1}),
}

_tick("Xlsxwriter workbook + formats init")

# --- Write Header Row ---
worksheet.write(0, 0, "", header_format)
for col_idx, week in enumerate(weeks, start=1):
    worksheet.write(0, col_idx, week, header_format)

_tick("Header row write")

# --- Write Data Rows with Grouping ---
for row_idx, (row_name, values, level) in enumerate(rows_data, start=1):
    # Set row outline level for grouping
    worksheet.set_row(row_idx, None, None, {'level': level, 'collapsed': level > 1})

    # Write row name
    worksheet.write(row_idx, 0, row_name, level_formats.get(level, left_format))

    # Write values
    for col_idx, val in enumerate(values, start=1):
        worksheet.write(row_idx, col_idx, val, cell_format)

_tick("All data rows write")

# --- Column Widths ---
worksheet.set_column(0, 0, 34)  # Column A
worksheet.set_column(1, len(weeks), 14)  # Data columns

# --- Freeze Panes ---
worksheet.freeze_panes(1, 1)

# --- Outline Settings ---
worksheet.outline_settings(True, False, False, False)

_tick("Sheet settings (width/freeze/outline)")

workbook.close()
_tick("Workbook close (file write flush)")


print(f"\n✅ SUCCESS! Summary saved to: {OUTPUT_FILE}")
print(f"   📊 Total rows: {len(rows_data):,}")
print(f"   📅 Weeks covered: {len(weeks)}")
print(f"   ⏱️  Total time: {(_time.perf_counter() - _T0):.2f}s")


KeyboardInterrupt: 